In [2]:
import os
openai_api_key = os.environ.get("API_KEY")

In [56]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = TextLoader('structura.txt')
documents = loader.load()


In [57]:
# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=5)

# Split your docs into texts
docs = text_splitter.split_documents(documents)

# Get embedding engine ready
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

# Embedd your texts
db = FAISS.from_documents(docs, embeddings)

In [58]:
retriever = db.as_retriever()
prueba = retriever.get_relevant_documents("Lista de camareros")
prueba

[Document(page_content='NULL REFERENCES "camareros" ("ID") DEFERRABLE INITIALLY DEFERRED, "Fecha" date NOT NULL)', metadata={'source': 'structura.txt'}),
 Document(page_content='NOT NULL, "camarero_id" integer NOT NULL REFERENCES "camareros" ("ID") DEFERRABLE INITIALLY DEFERRED, "receptor_id" integer NOT NULL REFERENCES', metadata={'source': 'structura.txt'}),
 Document(page_content='CREATE TABLE "camareros" ("ID" integer NOT NULL PRIMARY KEY AUTOINCREMENT, "Nombre" varchar(100) NOT NULL, "Apellidos" varchar(100) NOT NULL, "Email"', metadata={'source': 'structura.txt'}),
 Document(page_content='integer NOT NULL REFERENCES "camareros" ("ID") DEFERRABLE INITIALLY DEFERRED, "IDLPedido" integer NOT NULL REFERENCES "lineaspedido" ("ID") DEFERRABLE', metadata={'source': 'structura.txt'})]

In [59]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0, openai_api_key=openai_api_key)

In [60]:
# The LangChain component we'll use to get the documents
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())

In [61]:
from langchain.prompts import PromptTemplate

# Create our template
template = """
Eres una IA experta en traducir texto a SQL:   

Podrias crar una consulta SQL con la {consulta}. Solo necesito la consulta sin explicaciones.
En una linea.
"""

p = PromptTemplate(
    template=template,
    input_variables=["consulta"]
)

In [ ]:
qa.run(p.format(consulta="Dame la lista de camareros") )

'SELECT * FROM camareros;'

In [ ]:
qa.run(p.format(consulta="Lista de los 100 ultimos ticket"))

'SELECT * FROM ticket ORDER BY ID DESC LIMIT 100;'

In [62]:
qa.run(p.format(consulta="Dame la informacion completa de la mesa Z1"))

"SELECT * FROM infmesa WHERE UID = 'Z1';"

In [16]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI
import json

llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=openai_api_key, temperature=0)

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Example Input: {input}\nExample Output: {output}",
)

with open("str_tes.json", "r") as f:
    result = json.loads(f.read())

In [17]:
# SemanticSimilarityExampleSelector will select examples that are similar to your input by semantic meaning

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    result, 
    
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(openai_api_key=openai_api_key), 
    
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS, 
    
    # This is the number of examples to produce.
    k=4
)

In [18]:
similar_prompt = FewShotPromptTemplate(
    # The object that will help select examples
    example_selector=example_selector,
    
    # Your prompt
    example_prompt=example_prompt,
    
    # Customizations that will be added to the top and bottom of your prompt
    prefix="Traduce el texto a una consulta sql.",
    suffix="Input: {text}\nOutput:",
    
    # What inputs your prompt will receive
    input_variables=["text"],
)

In [94]:
# Select a noun!
text = "Dame la lista de camareros"

print(similar_prompt.format(text=text))

Traduce el texto a una consulta sql.

Example Input: Obtener el nombre, apellido y email de todos los camareros
Example Output: SELECT Nombre, Apellidos, Email FROM camareros

Example Input: Obtener información de los camareros
Example Output: SELECT ID, Nombre, Apellido, Autorizado FROM camareros;

Example Input: ¿Cuál es el ID, nombre, apellido y autorización de los camareros?
Example Output: SELECT ID, Nombre, Apellido, Autorizado FROM camareros;

Example Input: Obtener los nombres de todos los camareros autorizados
Example Output: SELECT Nombre FROM camareros WHERE Autorizado = 1

Input: Dame la lista de camareros
Output:


In [95]:
llm(similar_prompt.format(text="dame una lista de 10 mesas que contengan nulos"))

"WITH latest_unique_historialnulos AS (SELECT * FROM (SELECT historialnulos.*, lp.IDArt, lp.Estado, lp.UID FROM historialnulos INNER JOIN lineaspedido as lp ON lp.ID = historialnulos.IDLPedido GROUP BY UID) t ORDER BY ID DESC LIMIT 10) SELECT infmesa.Hora, camareros.Nombre AS camarero_nombre, mesas.Nombre AS mesa_nombre, infmesa.Fecha, COUNT(CASE WHEN lineaspedido.Estado = 'A' THEN 1 END) AS nulos, COUNT(CASE WHEN lineaspedido.Estado = 'P' THEN 1 END) AS pedidos, COUNT(CASE WHEN lineaspedido.Estado = 'C' THEN 1 END) AS cobrados FROM infmesa INNER JOIN latest_unique_historialnulos ON infmesa.UID = latest_unique_historialnulos.UID INNER JOIN pedidos ON pedidos.UID = infmesa.UID INNER JOIN lineaspedido ON lineaspedido.IDPedido = pedidos.ID INNER JOIN camareros ON camareros.ID = infmesa.IDCam INNER JOIN mesas ON mesas.ID = CAST(SUBSTR(infmesa.UID, 1, INSTR(infmesa.UID, '-') - 1) AS INT) WHERE lineaspedido.Estado = 'A' GROUP BY infmesa.UID, infmesa.Hora, camareros.Nombre, mesas.Nombre, infm

In [96]:
llm(similar_prompt.format(text="Dame la informacion de la mesa S16 del 1 de mayo del 2023 a las nueve y 19"))

"WITH ic AS (SELECT Hora, Fecha, IDCam, UID, CAST(SUBSTR(infmesa.UID, 1, INSTR(infmesa.UID, '-') - 1) AS INT) as IDMesa FROM infmesa WHERE Hora = '09:19' AND Fecha = '2023-05-01') SELECT ic.Hora, cm.Nombre AS cam_mesa, mesas.Nombre AS mesa_nombre, ic.Fecha, lp.Precio, Count(lp.IDArt) as Can, lp.Descripcion, Count(lp.IDArt) * lp.Precio as Total, lp.Estado, p.ID as IDPedido, cp.Nombre AS cam_pedido, p.Hora as hora_pedidos FROM ic INNER JOIN mesas ON mesas.ID = ic.IDMesa AND mesas.Nombre = 'S16' INNER JOIN pedidos p ON p.UID = ic.UID INNER JOIN lineaspedido lp ON  lp.IDPedido = p.ID INNER JOIN camareros cp ON cp.ID = p.IDCam INNER JOIN camareros cm ON cm.ID=ic.IDCam GROUP BY p.ID, lp.IDArt, lp.Descripcion, lp.Estado;"

In [97]:
llm(similar_prompt.format(text="Activa al camarero manuel"))

"UPDATE camareros SET Activo = 1 WHERE Nombre = 'Manuel'; SELECT Nombre, Apellidos, Autorizado from camareros WHERE Activo = 1"

In [98]:
llm(similar_prompt.format(text="Borra a manuel rodriguez"))

"UPDATE camareros SET Activo = 0 WHERE Nombre = 'Manuel' AND Apellidos = 'Rodriguez'; SELECT Nombre, Apellidos, Autorizado from camareros WHERE Activo = 1"

In [23]:
from typing import Optional, Type

from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from langchain.schema import BaseModel, Field
from langchain.tools import BaseTool
import sqlite3
from sqlite3 import Error
from typing import Optional, Type

# You can provide a custom args schema to add descriptions or custom validation

class SearchSchema(BaseModel):
    query: str = Field(description="should be a SQL query")
   

class ExecSQLTools(BaseTool):
    name = "exec_sql"
    description = "Useful when you need Execute a UPDATE, INSERT or DELETE SQL query"
    args_schema: Type[SearchSchema] = SearchSchema

    

    def create_connection(self):
        conn = None;
        try:
            conn = sqlite3.connect(db_file)
        except Error as e:
            print(e)
        return conn

    def execute_sql(self, conn, sql):
        try:
            c = conn.cursor()
            c.execute(sql)
            conn.commit()
        except Error as e:
            print(e)
        finally:
            if conn:
                conn.close()

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        conn = self.create_connection()
        if conn is not None:
            self.execute_sql(conn, query)
            return "SQL query executed successfully"
        else:
            return "Error! Cannot create the database connection."

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        return self._run(query, run_manager)
    
class QuerySQLTools(BaseTool):
    name = "query_sql"
    description = "Useful when you need Execute a SELECT SQL query"
    args_schema: Type[SearchSchema] = SearchSchema

    

    def create_connection(self):
        conn = None;
        try:
            conn = sqlite3.connect(db_file)
        except Error as e:
            return e
        return conn

    def execute_query(self, conn, sql):
        try:
            c = conn.cursor()
            c.execute(sql)
            return [dict(zip([column[0] for column in c.description], row)) 
                for row in c.fetchall()]
        except Error as e:
            return e
        finally:
            if conn:
                conn.close()

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        conn = self.create_connection()
        if conn is not None:
            results = self.execute_query(conn, query)
            return results
        else:
            return "Error! Cannot create the database connection."

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        return self._run(query, run_manager)
    

In [27]:
from langchain.agents import initialize_agent
from langchain.agents.types import AgentType
from langchain.chains import LLMChain
from langchain.tools import Tool
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

db_file="../../../testtpv.sqlite3"
llm_chain = LLMChain(llm=llm, prompt=prompt)

llm_tool = Tool(
   name="Modelo del lenguaje",
   func=llm_chain.run,
   description='use this tool for genera purpose queries an logic'
)


tools = [ExecSQLTools(), QuerySQLTools(), llm_tool]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, max_iterations=3, verbose=True)

In [24]:

agent.run(similar_prompt.format(text="Autoriza a manuel, cristian. desautoriza a los demas"))



> Entering new AgentExecutor chain...
We need to update the "Autorizado" column for Manuel and Cristian to 1 and for everyone else to 0.
Action: exec_sql
Action Input: 
UPDATE camareros SET Autorizado = CASE WHEN Nombre IN ('Manuel', 'Cristian') THEN 1 ELSE 0 END;
Observation: SQL query executed successfully
Thought:Now we need to retrieve the updated data to confirm the changes.
Action: query_sql
Action Input: SELECT Nombre, Apellidos, Autorizado from camareros
Observation: [{'Nombre': 'Bart', 'Apellidos': 'Simpson', 'Autorizado': 0}, {'Nombre': 'Eloy', 'Apellidos': 'Ruiz', 'Autorizado': 0}, {'Nombre': 'Raul', 'Apellidos': 'Blanco', 'Autorizado': 0}, {'Nombre': 'Agustin', 'Apellidos': 'Martinez', 'Autorizado': 0}, {'Nombre': 'Alejandro', 'Apellidos': 'Fernandez', 'Autorizado': 0}, {'Nombre': 'Manuel', 'Apellidos': 'Rodriguez', 'Autorizado': 1}, {'Nombre': 'Omar', 'Apellidos': '', 'Autorizado': 0}, {'Nombre': 'José Luis', 'Apellidos': 'Rueda', 'Autorizado': 0}, {'Nombre': 'Daniel', '

'The "Autorizado" column for Manuel and Cristian has been updated to 1, and for everyone else to 0.'

In [30]:
prompt = input()
agent.run(similar_prompt.format(text=prompt))



> Entering new AgentExecutor chain...
This input is similar to the first example input, so I should use query_sql to execute a SELECT query.
Action: query_sql
Action Input: "SELECT Nombre FROM camareros WHERE Autorizado = 1"
Observation: [{'Nombre': 'Manuel'}, {'Nombre': 'Cristian'}]
Thought:The query returned the names of authorized waiters, so I need to modify it to include the ID and last name as well.
Action: query_sql
Action Input: "SELECT ID, Nombre, Apellido FROM camareros WHERE Autorizado = 1"
Observation: no such column: Apellido
Thought:I made a mistake in the query, the column name should be "Apellido" instead of "Apellidos".
Action: query_sql
Action Input: "SELECT ID, Nombre, Apellido FROM camareros WHERE Autorizado = 1"
Observation: no such column: Apellido
Thought:

> Finished chain.


'Agent stopped due to iteration limit or time limit.'